In [ ]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [ ]:
gebruikers_df = pd.read_csv(r'..\..\data\gebruikersinformatie.csv', delimiter = ';')
evenement_df = pd.read_csv(r'..\..\data\evenementinformatie.csv', delimiter = ';')
aanwezigheid_df = pd.read_csv(r'..\..\data\evenementaanwezigheid.csv', delimiter = ';')

all_df = [gebruikers_df, evenement_df, aanwezigheid_df]

In [ ]:
datetime_cols = ['RegistratieDatum', 'LaatsteLogin', 'EvenementDatum', 'BerichtDatum']
id_cols = ['GebruikerID', 'EvenementID', 'OrganisatorID', 'BerichtID']

for df in all_df:
    for col in df.columns:
        if col in datetime_cols:
            df[col] = pd.to_datetime(df[col], dayfirst = True)

        elif col in id_cols:
            df[col] = df[col].str[1:].astype(int)

In [ ]:
# Remove invalid gebruikers
for col in gebruikers_df.columns:
    gebruikers_df = gebruikers_df[~(gebruikers_df[col].isna())]

gebruikers_df = gebruikers_df[~(gebruikers_df['Lidmaatschapstype'] == 'x')]

gebruikers_df

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------------------------------
# Alleen 'Bijgewoond'
filtered_events = aanwezigheid_df[aanwezigheid_df['Aanwezigheidsstatus'] == 'Bijgewoond'].copy()

# # Alles behalve 'Afwezig' (dus 'Bijgewoond' en 'Geregistreerd')
# filtered_events = aanwezigheid_df.copy()
# filtered_events = filtered_events[~(filtered_events['Aanwezigheidsstatus'] == 'Afwezig')]

#------------------------------------------------------------------------------------------------------------------------------------------------------

# Group event attendance data by GebruikerID
events_per_user = (
    filtered_events.groupby('GebruikerID')['EvenementID']
    .apply(list)
    .reset_index()
    .rename(columns = {'EvenementID': 'Geregistreerde evenementen'})
)

# Merge gebruikers_df with the attended events
gebruikers_df = gebruikers_df.merge(events_per_user, on = 'GebruikerID', how = 'left')

# Fix typo's
gebruikers_df['Studierichting'] = gebruikers_df['Studierichting'].replace('Data Engineering', 'Data engineering')

# Delete entries with no evenementen bijgewoond
gebruikers_df = gebruikers_df[~(gebruikers_df['Geregistreerde evenementen'].isna())]

gebruikers_df

In [ ]:
# Select demographic features for similarity
predictors = ['Leeftijd', 'Studierichting', 'Studiejaar', 'Lidmaatschapstype']

# Make dummies
dummies_df = pd.get_dummies(gebruikers_df[predictors])

# Train a KNN model for finding similar users
knn = NearestNeighbors(n_neighbors = 10)
knn.fit(dummies_df)

In [ ]:
# Empty template df
template_df = pd.DataFrame({
    'Leeftijd': [0],
    'Studierichting_Data engineering': False,
    'Studierichting_Human computer interaction': False,
    'Studierichting_Security & Cloud': False,
    'Studierichting_Software engineering': False,
    'Studiejaar_Derdejaars': False,
    'Studiejaar_Eerstejaars': False,
    'Studiejaar_Tweedejaars': False,
    'Studiejaar_Vierdejaars': False,
    'Lidmaatschapstype_Premium': False,
    'Lidmaatschapstype_Regulier': False,
})

# New user data
new_user_df = pd.DataFrame({
    'Leeftijd': [25],
    'Studierichting': ['Software engineering'],
    'Studiejaar': [3],
    'Lidmaatschapstype': ['Premium']
})

# Define variable column names
studierichting_col = f"Studierichting_{new_user_df.loc[0, 'Studierichting']}"
studiejaar_col = f"Studiejaar_{['Eerstejaars', 'Tweedejaars', 'Derdejaars', 'Vierdejaars'][new_user_df.loc[0, 'Studiejaar'] - 1]}"
lidmaatschap_col = f"Lidmaatschapstype_{new_user_df.loc[0, 'Lidmaatschapstype']}"

# Update values in template_df
template_df.loc[0, 'Leeftijd'] = new_user_df.loc[0, 'Leeftijd']
template_df.loc[0, [studierichting_col, studiejaar_col, lidmaatschap_col]] = True

new_user_df = template_df

new_user_df

In [ ]:
# Find similar users
distances, indices = knn.kneighbors(new_user_df)

# Get similar users' event preferences
similar_users = gebruikers_df.iloc[indices[0]]
similar_users

In [ ]:
# Get recommended event ids and write into dataframe
recommended_event_ids = similar_users['Geregistreerde evenementen'].sum()
recommended_event_ids_df = pd.DataFrame({'EvenementID': recommended_event_ids})

# Get full information on these events by merging with evenementinformatie
recommended_events = recommended_event_ids_df.merge(evenement_df, on = 'EvenementID', how = 'left')

# Count how often each type appears
recommended_events_type_counts = recommended_events['EvenementType'].value_counts()

recommended_events_type_counts